# Predição do Arquivo Players
Este notebook é dedicado à implementação e análise de um modelo de classificação utilizando o algoritmo Random Forest para prever a probabilidade de jogadores marcarem gols com base em suas características, como desempenho do jogador, números de gols e outras métricas. O objetivo deste notebook é fornecer uma ferramenta para previsões, que pode ser usada para previsões de resultados.

## Objetivo
O objetivo desse notebook é fornecer previsões se um jogador marcará um gol em uma partida. Através de análise dos dados e do treinamento de um mondelo Randon Forest, buscando identificar padrões e caractéristicas que mais influenciam o desempenho dos jogadores.

## Como usar Este Notebook


1.   **Configuração do Ambiente:**
      * **Google Colab:** No Google Colab, será necessário fazer o upload das tabelas para o Google Drive e montar o drive no notebook.
      * **Localmente:** Se for rodar o notebook localmente, é necessário baixar as tabelas e colocá-las no mesmo diretório do notebook ou ajustar os caminhos dos arquivos conforme necessário.


2.   **Instalação de Dependências:**
       * Certifique-se de que todas as bibliotecas necessárias estão instaladas. Você pode instalar as dependências utilizando o seguinte comando:


```
 !pip install -r requirements.txt
```

3.   **Execução do Notebook:**
      * Siga as células de código sequencialmente para garantir que todas as etapas sejam executadas corretamente. Cada seção do notebook está organizada para facilitar a compreensão e a análise dos dados.

##Nesse Notebook Será Abordado
1. **Preparação e Treinamento de Dados:**
      * Seleção e preparação das features e da variável target a partir do dataset, incluindo o tratamento de dados ausentes e a padronização das variáveis.
      * Implementação do modelo Random Forest para classificação dos jogadores com base em seu desempenho.

           
2. **Implementação de Função de Previsão:**
     * Desenvolvimento de uma função que permite prever a probabilidade de um jogador específico marcar um gol.
3. **Avaliação do Modelo:**
      * Cálculo de métricas de desempenho, como acurácia, precisão, recall e F1-score, para avaliar a eficácia do modelo.
      * Visualização da matriz de confusão para compreender melhor as previsões do modelo.


# Dependências
Para rodar o notebook de forma local, é recomendado que inicie uma venv (ambiente virtual) e instale as dependências.

Se estiver utilizando o Google Colab, pule esta etapa.

# Importando bibliotecas
Aqui é importado as dependências necessárias para a executação do projeto.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, balanced_accuracy_score
import matplotlib.pyplot as plt

# Carregando o dataset
Feita a importação do arquivo para leitura dos dados.

In [2]:
df = pd.read_csv('../../notebooks/data/tratado/players_tratado.csv')

In [ ]:
df.head(1)

In [ ]:
# Ajuste para criação da tabela de id e full_name de acordo com o estilo do código do usuário

# Resetando o índice para criar uma coluna de 'id'
player_table = df.reset_index()[['index', 'full_name']]
player_table.columns = ['id', 'table_name']

# Exibindo a tabela ajustada no Jupyter Notebook
display(player_table)
# Salvando a tabela em CSV
player_table.to_csv('player_table.csv', index=False)

## Treinamento do modelo


In [5]:
# Selecionar as features e a target
df['target'] = df['goals_overall'].apply(lambda x: 1 if x > 0 else 0)
X = df.drop(columns=['goals_overall', 'full_name', 'target'])  # Remove as colunas desnecessárias
y = df['target']  # Define a coluna 'target' como alvo

# Dividir os dados em conjunto de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=42)

# Treinar o modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Padronizar os dados com a mesma escala
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Aplica a padronização nos dados de treino
X_test = scaler.transform(X_test)  # Aplica a mesma padronização nos dados de teste

# Treinar o modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


columns_to_remove = ['goals_overall', 'full_name', 'target']
columns_to_remove = [col for col in columns_to_remove if col in df.columns]
original_columns = df.drop(columns=columns_to_remove).columns

# Seleciona apenas as colunas numéricas
X_numeric = X.select_dtypes(include=[np.number])

scaler = StandardScaler()
imputer = SimpleImputer()

X_imputed = imputer.fit_transform(X_numeric)
X_scaled = scaler.fit_transform(X_imputed)


## Seleção das principais features

Nessa etapa, utilizamos o método Random Forest para identificar as 10 principais features que o modelo utiliza para prever a probabilidade de um jogador marcar um gol, além de determinar a ordem de importância de cada uma delas. Esse processo é fundamental para entender quais variáveis mais influenciam no desempenho dos jogadores, garantindo um modelo mais rápido e eficiente para futuras previsões. Esse processo é fundamental para entender quais fatores têm maior impacto no resultado final, permitindo uma tomada de decisões mais assertiva na análise de desempenho dos jogadores.

Além disso, utilizamos o método StandardScaler para normalizar as variáveis numéricas do conjunto de dados, garantindo que todas as features tenham a mesma escala. Isso é importante porque algoritmos como o Random Forest podem ser sensíveis a variáveis com diferentes magnitudes, o que pode influenciar negativamente o desempenho do modelo. A normalização permite que o modelo trate todas as variáveis de forma equilibrada, evitando vieses e melhorando a precisão das previsões.

O gráfico foi gerado para visualizar a importância das principais features do modelo, facilitando a interpretação dos resultados. Após calcular a importância de cada variável com o Random Forest, criamos um DataFrame organizado que exibe as variáveis em ordem de relevância. A seguir, foi utilizada uma visualização em formato de gráfico de barras horizontais para destacar as 10 features mais importantes, invertendo o eixo y para que as mais relevantes apareçam no topo. Esse gráfico oferece uma representação clara e intuitiva de quais variáveis têm maior impacto no desempenho do modelo.

In [ ]:
# Verifica a importância das features do modelo
importances = model.feature_importances_

# Cria um DataFrame para organizar as importâncias das features
feature_importances = pd.DataFrame({
    'Feature': df.drop(columns=['goals_overall', 'full_name', 'target']).columns,
    'Importance': importances
})

# Ordena as features pela importância
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# Exibe as 10 principais features
print(feature_importances.head(10))

# Opcional: visualiza as importâncias das features
plt.figure(figsize=(10, 6))
plt.title('Importância das Principais Features')
plt.barh(feature_importances['Feature'].head(10), feature_importances['Importance'].head(10))
plt.gca().invert_yaxis()  # Inverte o eixo y para que a feature mais importante apareça no topo
plt.show()

## Organização dos dados

Por se tratar de um modelo que prevê uma probabilidade de um jogador marcar um gol, o grupo optou por manter uma proporção de 60% de dados de treinamento e 40% de dados de teste.

# Implementação de Função de Previsão
Nesta seção, desenvolvemos uma função personalizada que permite prever a probabilidade de um jogador específico marcar um gol, usando o modelo treinado. A função verifica se o jogador está presente no dataset, prepara seus dados de forma consistente com o modelo e calcula a probabilidade da previsão.

**Desenvolvimento da Função predict_player_goal(player_id):**

A função predict_player_goal é criada para prever a probabilidade de um jogador marcar um gol. Ela realiza uma série de verificações e pré-processamentos nos dados do jogador antes de fazer a previsão com o modelo.

In [ ]:
# Função para prever a probabilidade de gol
def predict_player_goal(table_name):
    global imputer, scaler, model, original_columns, df

    # Verifica se o nome do jogador existe na tabela player_table
    if table_name not in player_table['table_name'].values:
        return f"Jogador com nome '{table_name}' não encontrado."

    # Obtém o id do jogador
    player_id = player_table.loc[player_table['table_name'] == table_name, 'id'].values[0]

    # Verifica se o nome completo (full_name) existe no DataFrame original
    if table_name not in df['full_name'].values:
        return f"Jogador com nome completo '{table_name}' não encontrado no DataFrame original."

    # Seleciona os dados do jogador usando o full_name
    player_data = df[df['full_name'] == table_name].copy()

    # Remove as colunas desnecessárias, verifica se elas existem
    columns_to_drop = ['goals_overall', 'target', 'full_name']
    columns_to_drop = [col for col in columns_to_drop if col in player_data.columns]
    player_data = player_data.drop(columns=columns_to_drop)

     # Realinhar as colunas do player_data para que correspondam ao original_columns
    player_data = player_data.reindex(columns=original_columns, fill_value=0)

    # Seleciona apenas as colunas numéricas
    player_data_numeric = player_data.select_dtypes(include=[np.number])


    # Imputação de dados
    player_data_imputed = imputer.transform(player_data_numeric)

     # Padronização dos dados
    player_data_scaled = scaler.transform(player_data_imputed)

    # Previsão da probabilidade de marcar um gol
    probabilidade = model.predict_proba(player_data)[:, 1]

    return f"A probabilidade do jogador '{table_name}' (ID: {player_id}) marcar um gol é de {probabilidade[0]*100:.2f}%."

# Exemplo de uso
table_name = "Paulinho"
print(predict_player_goal(table_name))


Nesta etapa do processo, utilizamos um conjunto de hiperparâmetros específicos para otimizar o desempenho do modelo RandomForestClassifier. Os hiperparâmetros ajustados foram:

- n_estimators: número de árvores na floresta, definido como 100.
- max_depth: profundidade máxima de cada árvore, definido como sem limite (None).
- min_samples_split: número mínimo de amostras necessárias para dividir um nó, definido como 2.
- min_samples_leaf: número mínimo de amostras necessárias para estar em um nó folha, definido como 1.
- bootstrap: habilitação do bootstrap para amostragem dos dados, definido como True.

Além disso, aplicamos a validação cruzada com 5 divisões estratificadas (StratifiedKFold) para garantir que o modelo fosse avaliado de forma equilibrada, considerando a distribuição das classes da variável-alvo em cada um dos folds. O GridSearchCV foi utilizado para explorar e selecionar a melhor combinação desses hiperparâmetros, garantindo a escolha do modelo com o melhor desempenho em termos de acurácia.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold

# Definir a target como uma classificação binária (0 ou 1)
df_copy = df.copy()
df_copy['target_rf'] = df_copy['goals_overall'].apply(lambda x: 1 if x > 0 else 0)

# Remover as colunas desnecessárias
columns_to_remove_rf = ['goals_overall', 'full_name', 'target_rf']
columns_to_remove_rf = [col for col in columns_to_remove_rf if col in df_copy.columns]
X_rf = df_copy.drop(columns=columns_to_remove_rf)
y_rf = df_copy['target_rf']  # Target é a coluna 'target_rf'

# Salvar as colunas originais para futura referência
original_columns_rf = X_rf.columns

# Selecionar apenas colunas numéricas
X_numeric_rf = X_rf.select_dtypes(include=[np.number])

# Padronizar os dados com a mesma escala e tratar valores faltantes
scaler_rf = StandardScaler()
imputer_rf = SimpleImputer(strategy='mean')

X_imputed_rf = imputer_rf.fit_transform(X_numeric_rf)
X_scaled_rf = scaler_rf.fit_transform(X_imputed_rf)

# Definir os hiperparâmetros para o GridSearchCV
param_grid_rf = {
    'n_estimators': [200],
    'max_depth': [None],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [True]
}

# Instanciar o modelo RandomForest
model_rf = RandomForestClassifier(random_state=42)

# Aplicar a Validação Cruzada com 5 folds
cv_rf = StratifiedKFold(n_splits=5)

# Usar GridSearchCV para otimizar os hiperparâmetros
grid_search_rf = GridSearchCV(estimator=model_rf, param_grid=param_grid_rf, cv=cv_rf, n_jobs=-1, verbose=2, scoring='accuracy')

# Ajustar o modelo aos dados
grid_search_rf.fit(X_scaled_rf, y_rf)

# Melhor modelo com os melhores hiperparâmetros
best_model_rf = grid_search_rf.best_estimator_
best_params_rf = grid_search_rf.best_params_

print(f"Melhores hiperparâmetros: {best_params_rf}")

# Função para prever a probabilidade de gol com o melhor modelo encontrado
def predict_player_goal_rf(table_name):
    global imputer_rf, scaler_rf, best_model_rf, original_columns_rf, df_copy

    # Verifica se o nome do jogador existe na tabela player_table
    if table_name not in player_table['table_name'].values:
        return f"Jogador com nome '{table_name}' não encontrado."

    # Obtém o id do jogador
    player_id_rf = player_table.loc[player_table['table_name'] == table_name, 'id'].values[0]

    # Verifica se o nome completo (full_name) existe no DataFrame original
    if table_name not in df_copy['full_name'].values:
        return f"Jogador com nome completo '{table_name}' não encontrado no DataFrame original."

    # Seleciona os dados do jogador usando o full_name
    player_data_rf = df_copy[df_copy['full_name'] == table_name].copy()

    # Remove as colunas desnecessárias, verifica se elas existem
    columns_to_drop_rf = ['goals_overall', 'target_rf', 'full_name']
    columns_to_drop_rf = [col for col in columns_to_drop_rf if col in player_data_rf.columns]
    player_data_rf = player_data_rf.drop(columns=columns_to_drop_rf)

    # Realinhar as colunas do player_data para que correspondam ao original_columns_rf
    player_data_rf = player_data_rf.reindex(columns=original_columns_rf, fill_value=0)

    # Seleciona apenas as colunas numéricas
    player_data_numeric_rf = player_data_rf.select_dtypes(include=[np.number])

    # Imputação de dados
    player_data_imputed_rf = imputer_rf.transform(player_data_numeric_rf)

    # Padronização dos dados
    player_data_scaled_rf = scaler_rf.transform(player_data_imputed_rf)

    # Previsão da probabilidade de marcar um gol
    probabilidade_rf = best_model_rf.predict_proba(player_data_scaled_rf)[:, 1]

    return f"A probabilidade do jogador '{table_name}' (ID: {player_id_rf}) marcar um gol é de {probabilidade_rf[0]*100:.2f}%."

# Exemplo de uso
table_name_rf = "Paulinho"
print(predict_player_goal_rf(table_name_rf))


#  Avaliação do Modelo

Após o treinamento, é essencial avaliar o desempenho do modelo para verificar sua eficácia na classificação correta dos jogadores que marcaram gols. Nesta seção, calculamos métricas de desempenho como acurácia, acurácia balanceada, precisão, recall e F1-score. Além disso, visualizamos a matriz de confusão para entender as previsões do modelo. Dessa forma, ao analisar as métricas conseguimos avaliar de forma eficiente a execução do modelo, oque pode nos revelar se o conjunto de dados é satisfatório ou não para a tarefa proposta. Se o desempenho do modelo não for adequado, pode ser necessário realizar ajustes, como a coleta de mais dados, o balanceamento das classes ou a otimização de hiperparâmetros. Com esses insights, é possível aprimorar o modelo ou refinar a abordagem adotada, garantindo resultados mais precisos e confiáveis no futuro.

**Cálculo das Métricas de Desempenho:**

Utilizamos as previsões do modelo no conjunto de teste para calcular as principais métricas de desempenho. Essas métricas ajudam a entender a precisão do modelo e como ele lida com os falsos positivos e falsos negativos.

In [ ]:
# Imputar e padronizar os dados de teste
X_test_imputed = imputer.transform(X_test)  # X_test já é um ndarray
X_test_scaled = scaler.transform(X_test_imputed)

# Previsão no conjunto de teste
y_pred = model.predict(X_test_scaled)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_pred)

# Calcular a acurácia balanceada
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

# Exibir as métricas
print(f"Acurácia do modelo: {accuracy * 100:.2f}%")
print(f"Acurácia balanceada do modelo: {balanced_accuracy * 100:.2f}%")
print("Relatório de Classificação:\n", classification_report(y_test, y_pred))


## Avaliação das métricas com validação cruzada e hiperparâmetros
Os resultados apresentados com acurácia, precisão e recall de 1.00 indicam que o modelo pode estar sofrendo de overfitting, o que significa que ele está memorizando os padrões dos dados de treino e não generalizando bem para novos dados. Isso é um problema porque um desempenho perfeito nas métricas dificilmente reflete a realidade de modelos de aprendizado de máquina e pode sugerir uma falta de variabilidade nos dados ou uma má configuração na validação cruzada.

Portanto, é essencial revisar a estratégia de validação, ajustar melhor os hiperparâmetros e garantir que os dados estejam balanceados e diversos. O objetivo é obter um modelo que não apenas tenha bom desempenho em treino, mas que também seja robusto o suficiente para fazer previsões confiáveis em dados novos.








## Curva ROC e AUC para o Random Forest sem hiperparâmetros e validação cruzada
A **Curva ROC (Receiver Operating Characteristic)** é uma ferramenta gráfica que ilustra a capacidade de um modelo de classificação binária em distinguir entre classes positivas e negativas. A curva traça a relação entre a **Taxa de Verdadeiros Positivos (TPR)** e a **Taxa de Falsos Positivos (FPR)** para diferentes limiares de decisão.

O **AUC (Area Under the Curve)** mede a área sob a Curva ROC, fornecendo uma única métrica que reflete a capacidade do modelo de classificar corretamente as amostras. Um AUC próximo de 1 indica um modelo excelente, enquanto um AUC de 0.5 sugere que o modelo não é melhor que uma escolha aleatória.



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np

# Valores extraídos da matriz de confusão
TP = 133
FP = 40
TN = 165
FN = 15

# Calcula a taxa de verdadeiros positivos (TPR) e a taxa de falsos positivos (FPR)
# TPR = TP / (TP + FN)
# FPR = FP / (FP + TN)
tpr = TP / (TP + FN)
fpr = FP / (FP + TN)

# Como uma curva ROC requer múltiplos limiares para ser desenhada, normalmente precisaríamos das previsões de probabilidade.
# Neste caso, vamos usar uma simplificação para plotar a curva com base nos valores da matriz de confusão.

# Simulando os pontos da curva ROC com base nos valores da matriz de confusão
fpr_values = np.array([0.0, fpr, 1.0])
tpr_values = np.array([0.0, tpr, 1.0])

# Calcula a AUC (Área sob a Curva)
roc_auc = auc(fpr_values, tpr_values)

# Plotando a curva ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr_values, tpr_values, color='blue', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')  # Linha aleatória
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

# Exibe a AUC calculada
print(f"AUC: {roc_auc:.2f}")





### Resultados

Após gerar a Curva ROC e calcular o AUC, os seguintes resultados foram obtidos:

- **Curva ROC**: A curva mostra a taxa de verdadeiros positivos (TPR) em função da taxa de falsos positivos (FPR) para diferentes limiares de decisão. O formato da curva fornece uma visão sobre o desempenho do modelo de Regressão Logística em discriminar entre classes positivas e negativas.

- **AUC (Area Under the Curve)**: O valor calculado foi de **0.85**, indicando que o modelo possui uma capacidade de classificação. Um AUC próximo de 1 sugere que o modelo é muito eficaz em distinguir corretamente entre as classes.

### Conclusão

A avaliação do modelo usando a Curva ROC e AUC mostrou que o modelo tem um desempenho na tarefa de classificação:

- **Desempenho de Classificação**: A curva ROC, que se aproxima do canto superior esquerdo do gráfico, demonstra que o modelo possui uma alta taxa de verdadeiros positivos em relação aos falsos positivos, o que é ideal para um classificador binário.

- **Qualidade do Modelo**: Com um AUC de **0.85**, o modelo demonstra boa precisão e habilidade em classificar corretamente as instâncias. Este resultado sugere que o modelo pode ser adequado para ser utilizado no problema em questão, dado o seu alto poder discriminativo.

#  Visualização da Matriz de Confusão do modelo sem hiperparâmetros e validação cruzada
Este código gera uma matriz de confusão para visualizar o desempenho do modelo de classificação em termos de suas previsões. Utilizando o Seaborn, uma biblioteca de visualização de dados do Python, o código cria um heatmap que exibe a matriz de confusão, onde as previsões corretas e incorretas do modelo são claramente indicadas. A matriz mostra a contagem de classificações corretas (verdadeiros positivos e negativos) e incorretas (falsos positivos e negativos) para as classes "Positivo" e "Negativo".

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Dados da matriz de confusão
cm_data = [[219, 83],
           [18, 209]]

# Visualizar a matriz de confusão usando seaborn heatmap
plt.figure(figsize=(8,6))  # Define o tamanho da figura
sns.heatmap(cm_data, annot=True, fmt='d', cmap='Blues', xticklabels=['Negativo', 'Positivo'], yticklabels=['Negativo', 'Positivo'])
plt.xlabel('Previsão')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()


#  Referências
Está é uma seção de referências com relação as bibliotecas que utilizamos ao longo deste arquivo

NUMPY. NumPy Documentation. Disponível em: https://numpy.org/doc/.

‌PANDAS. pandas documentation. Disponível em: https://pandas.pydata.org/docs/.

MATPLOTLIB. Matplotlib: Python plotting — Matplotlib 3.3.4 documentation. Disponível em: https://matplotlib.org/stable/index.html.

‌SCIKIT-LEARN. scikit-learn: machine learning in Python. Disponível em: https://scikit-learn.org/stable/.

‌SEABORN. seaborn: statistical data visualization — seaborn 0.9.0 documentation. Disponível em: <https://seaborn.pydata.org/>.